In [6]:
! python -m pip install -q pymilvus==0.2.10 towhee gradio opencv-python pillow

In [2]:
import pandas as pd

df = pd.read_csv('data/towhee/reverse_image_search.csv')
df.head()


,id,path
0,0,data/demo/KeyFramesC00_V00/C00_V0010/024860.jpg
1,1,data/demo/KeyFramesC00_V00/C00_V0010/022153.jpg
2,2,data/demo/KeyFramesC00_V00/C00_V0010/014370.jpg
3,3,data/demo/KeyFramesC00_V00/C00_V0010/015957.jpg
4,4,data/demo/KeyFramesC00_V00/C00_V0010/023130.jpg


In [7]:
import cv2
from towhee._types.image import Image

id_img = df.set_index('id')['path'].to_dict()

def read_images(results):
    imgs = []
    for re in results:
        path = id_img[re.id]
        imgs.append(Image(cv2.imread(path), 'BGR'))
    return imgs



In [8]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

connections.connect(host='127.0.0.1', port='19530')


def create_milvus_collection(collection_name, dim):
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
        FieldSchema(name='id', dtype=DataType.INT64,
                    descrition='ids', is_primary=True, auto_id=False),
        FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR,
                    descrition='embedding vectors', dim=dim)
    ]
    schema = CollectionSchema(fields=fields, description='text image search')
    collection = Collection(name=collection_name, schema=schema)

    # create IVF_FLAT index for collection.
    index_params = {
        'metric_type': 'L2',
        'index_type': "IVF_FLAT",
        'params': {"nlist": 512}
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection


collection = create_milvus_collection('text_image_search', 512)


MilvusException: <MilvusException: (code=2, message=Fail connecting to server on 127.0.0.1:19530. Timeout)>

In [8]:
!pip install grpcio==1.48.0

     |████████████████████████████████| 4.6 MB 1.3 MB/s eta 0:00:01
Reason for being yanked: Deadlock observed in Apache Beam.
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.2
    Uninstalling grpcio-1.47.2:
      Successfully uninstalled grpcio-1.47.2
